In [ ]:
# -------------------------------------
# Master script for Maritime Risk Model
#
#         Ryan Lloyd
#         22/10/2018
# -------------------------------------

# ------- USAGE --------
#
# 1. Requires list of AIS data and incidents
# 2. Requires some static model parameters
#
# ----------------------

In [1]:
# Load environments
import os
import sys
import arcgis
import arcpy
import pandas
import geopandas
import csv
import numpy
import matplotlib
import matplotlib.pyplot
import pylab
import glob
import zipfile
from arcpy.sa import *
from importlib import reload

In [2]:
# Input files
AIS123_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/AIS_123/" # OS SPECIFIC "/"
AIS005_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/AIS_005/" # OS SPECIFIC
incidents_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/incidents/" # OS SPECIFIC
AIS123_file = "files_123.txt"
AIS005_file = "files_005.txt"
incidents_file = "incidents.txt"
continent_polygon = "c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/continents/World_Continents.lyrx"

FileGDB_path = "c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/"
FileGDB_name = "risk_model.gdb"

In [3]:
# Variables (FN size etc)

AISjoin_field = "MMSI"
AISjoin_order = "ISODate"
fishnet_extent = [7, 10, -3, -7] # N,E,S,W
fishnet_size = [0.5, 0.5]   # width, height (degrees?)

In [3]:
# ------ DO NOT EDIT BELOW HERE ----------

In [4]:
# Add folder of python modules:
sys.path.append(r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_functions/")

In [51]:
import load_data
import create_FileGDB
import traffic
reload(load_data)
reload(create_FileGDB)
reload(traffic)

<module 'traffic' from 'C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_functions\\traffic.py'>

In [52]:
# Load local modules
from load_data import csv2shp, loadais, loadincidents, csv2dict
from create_FileGDB import creategdb
from traffic import ais2traffic

In [23]:
# Set up FileGDB
creategdb(FileGDB_path, FileGDB_name)
arcpy.env.workspace = FileGDB_path + FileGDB_name

FileGDB already exists


In [9]:
# Load in the AIS and incidents
incidentspts = csv2shp(incidents_root,incidents_file,FileGDB_path,FileGDB_name)
AIS123pts = csv2shp(AIS123_root, AIS123_file,FileGDB_path,FileGDB_name)

csv files converted to points:
incidents_022017.csv >> incidents_022017
csv files converted to points:
GoG123_20170201.csv >> GoG123_20170201
GoG123_20170202.csv >> GoG123_20170202
GoG123_20170203.csv >> GoG123_20170203


In [54]:
AIS123pts_test = AIS123pts[:1]
traffic = ais2traffic(FileGDB_path,AIS123pts_test,AISjoin_field,AISjoin_order,continent_polygon)

Output traffic files:
c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170201_offshore_traffic_lyr
['c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170201_offshore_traffic_lyr']


In [ ]:
# Create fishnet function

fishnet_extent = [7, 10, -3, -7] # N,E,S,W
# Within module that also has hexagons (create tesselation)
traffic = traffic_layers

def createfishnet(traffic, fishnet_extent, fishnet_size)
for f in traffic:
    OP_NAME = (traffic[len(root):len(root)+3]) + "_fishnet"
    arcpy.CreateFishnet_management(OP_NAME, "fishnet_extent[1] fishnet_extent[2]", "fishnet_extent[1] fishnet_extent[3]", fishnet_size[0],fishnet_size[1],"0","0","fishnet_extent[0] fishnet_extent[3]","LABELS",f,"POLYGON")

    # Add attribute with 0 value
arcpy.AddField_management('GoGFN2', "SA_3", "FLOAT","","5","","","NULLABLE","REQUIRED", "")
  # Create new dummy field of full shape area
arcpy.CalculateField_management("GoGFN2","SA_3", "!Shape_Area!") 
  # Clip GoG_FN to continents
arcpy.Clip_analysis("GoGFN2", "World_Continents", "landclip","")
# Create new field
arcpy.AddField_management('landclip', "SA_3", "FLOAT","","5","","","NULLABLE","REQUIRED", "")
# Find Sea Area
arcpy.CalculateField_management("landclip","SA_3", "math.pow(float(A),2)-!Shape_Area!", 'PYTHON3')
# Merge datasets
arcpy.Merge_management(['landclip',"GoGFN2"], "GoGFN_land3", "First")



In [ ]:
# Calculate attack density function

# How to update this as more data becomes availible? - see my adding proportionally notes
# Which attributes are needed?
# Going to need a 1-2-many for multiple attacks in cell -- relate?
arcpy.SpatialJoin_analysis("GoGFN_land3", "c:\\Users\\Geollect\\Documents\\ArcGIS\\Projects\\risk_2\\risk_2.gdb\\attacks_GoG_2_XYTableToPoint1","GoG_ACountCC7","JOIN_ONE_TO_ONE","KEEP_ALL","","COMPLETELY_CONTAINS")


### Attack density with several ips...
# add attack density = 0 field. then sum (kinda like iteratively)

# DIVIDE -- attacks/deg (deg^-1)
arcpy.AddField_management('GoG_ACountCC3', "attack_dens", "FLOAT","","5","","","NULLABLE","REQUIRED", "")
arcpy.CalculateField_management("GoG_ACountCC3","attack_dens", "!Join_Count!/(!SA_3!)", 'PYTHON3')

In [ ]:
# Line density....?